In [17]:
from services.db_service import DbService
from services.utils import get_banned_cards

decks_collection = DbService().deck_collection()

In [18]:
banned_cards = get_banned_cards()

pipeline = [
    {"$project": {"maindeck": 1, "sideboard": 1, "name": 1}},
    {"$project": {"_id": 1, "name": 1, "cards": {
        "$concatArrays": ["$maindeck", "$sideboard"]}}},
    {"$unwind": "$cards"},
    {"$group": {"_id": {"_id": "$_id", "name": "$name"},
                "found_cards": {"$addToSet": "$cards"}}},
    {"$match": {"found_cards": {"$in": banned_cards}}},
    {"$project": {"_id": "$_id._id"}}
]

In [19]:
decks = [deck["_id"] for deck in decks_collection.aggregate(pipeline)]

decks_collection.update_many({"_id": {"$in": decks}}, {
                             "$set": {"restricted": True}})

UpdateResult({'n': 251, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)